In [0]:
import requests
import os

# Make output directory
os.makedirs("db1b_market", exist_ok=True)

for year in range(1993, 2024):   # DB1B starts in 1993
    for q in [1,2,3,4]:
        url = (
            "https://www.transtats.bts.gov/Download_Bulk_Files.asp?"
            f"Table_ID=291&Has_Group=3&Is_Zipped=1&Year={year}&Qtr={q}"
        )

        r = requests.get(url)
        filename = f"db1b_market_{year}_q{q}.zip"

        with open(f"db1b_market/{filename}", "wb") as f:
            f.write(r.content)

        print("Downloaded", filename)

In [0]:
import zipfile

zip_path = "/Volumes/workspace/airline_origin_dest_data/d1b1/Origin_and_Destination_Survey_DB1BMarket_2025_1.zip"
extract_to = "/Volumes/workspace/airline_origin_dest_data/d1b1/unzipped_data"

# Create the folder (if needed)
dbutils.fs.mkdirs(extract_to)

# Unzip directly from the Volume to the Volume
with zipfile.ZipFile(zip_path, 'r') as z:
    for member in z.namelist():
        # Extract each file to the destination Volume path
        target_path = f"{extract_to}/{member}"
        with z.open(member) as source, open(target_path, "wb") as target:
            target.write(source.read())

print("Extraction complete.")

In [0]:
csv_path = "/Volumes/workspace/airline_origin_dest_data/d1b1/unzipped_data/Origin_and_Destination_Survey_DB1BMarket_2025_1.csv"

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_path)

df.display()

df.write.format("delta").mode("overwrite").saveAsTable(
    "workspace.airline_origin_dest_data.airline_data_table"
)
